# Windows Partitioning

## Prerrequisites

Install Spark and Java in VM

In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark 3.5.0
!wget -q https://apache.osuosl.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [ ]:
ls -l # check the .tgz is there

total 267684
drwxr-xr-x 1 root root      4096 Dec  6 14:35 sample_data/
-rw-r--r-- 1 root root 274099817 Oct 15 10:53 spark-3.3.1-bin-hadoop2.tgz


In [ ]:
# unzip it
!tar xf spark-3.5.0-bin-hadoop3.tgz

In [ ]:
!pip install -q findspark

Defining the environment

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Start Spark Session

---

In [ ]:
import findspark
findspark.init("spark-3.5.0-bin-hadoop3")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("Window Partitioning") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.3.1'

In [ ]:
spark

In [ ]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [ ]:
# Import sql functions
from pyspark.sql.functions import *

Download datasets

In [ ]:
!mkdir -p dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/characters.csv -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/employees.csv -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/salaries.csv -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/deptmanagers.csv -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/titles.csv -P /dataset
!ls /dataset

## Examples

### Window Partitioning

In [ ]:
employeesDF = spark.read.option("header", "true").csv("/dataset/employees.csv")
salariesDF = spark.read.option("header", "true").csv("/dataset/salaries.csv")
deptManagersDF = spark.read.option("header", "true").csv("/dataset/deptmanagers.csv")
titlesDF = spark.read.option("header", "true").csv("/dataset/titles.csv")
charactersDF = spark.read.option("inferSchema", "true").option("header", "true").csv("/dataset/characters.csv")

In [ ]:
titlesDF.show(2)

+------+--------+----------+----------+
|emp_no|   title| from_date|   to_date|
+------+--------+----------+----------+
| 10010|Engineer|1996-11-24|9999-01-01|
| 10020|Engineer|1997-12-30|9999-01-01|
+------+--------+----------+----------+
only showing top 2 rows



Get the last title for the employees

In [ ]:
titlesDF.filter(col("emp_no") == 10040).show(4)

+------+---------------+----------+----------+
|emp_no|          title| from_date|   to_date|
+------+---------------+----------+----------+
| 10040|       Engineer|1993-02-14|1999-02-14|
| 10040|Senior Engineer|1999-02-14|9999-01-01|
+------+---------------+----------+----------+



In [ ]:
# import library
from pyspark.sql.window import Window

In [ ]:
# get last title by date for each employee (similar to previous exercise in Joins)
byEmployee = Window.partitionBy("emp_no").orderBy(col("to_date").desc())
mostRecentJobTitlesDF = titlesDF.withColumn("datesOrder", row_number().over(byEmployee)) \
    .filter(col("datesOrder") == 1) \
    .select("emp_no", "title", "to_date")

mostRecentJobTitlesDF.filter(col("emp_no") == 10040).show(4)

+------+---------------+----------+
|emp_no|          title|   to_date|
+------+---------------+----------+
| 10040|Senior Engineer|9999-01-01|
+------+---------------+----------+



In [ ]:
# for the previous example we saw in joins, if we use windogs partitioning we do not need to do the previous step (filtering max date and then do the join over the same table)

Get the max salaries (three for example) for job title

In [ ]:
# we need salary data with title data joined first
bestPaidPerTitlerawDF = salariesDF.withColumn("salary", col("salary").cast("long")) \
    .join(mostRecentJobTitlesDF, (salariesDF.emp_no == mostRecentJobTitlesDF.emp_no) & (salariesDF.to_date == mostRecentJobTitlesDF.to_date)).drop("emp_no", "from_date", "to_date")

bestPaidPerTitlerawDF.filter(salariesDF.emp_no == 10040).show()
bestPaidPerTitlerawDF.show(3)

+------+---------------+
|salary|          title|
+------+---------------+
| 72668|Senior Engineer|
+------+---------------+

+------+---------------+
|salary|          title|
+------+---------------+
| 80324|       Engineer|
| 47017|       Engineer|
| 88806|Senior Engineer|
+------+---------------+
only showing top 3 rows



In [ ]:
# now we apply the window partitioning
byTitle = Window.partitionBy("title").orderBy(col("salary").desc())

bestPaidPerTitleDF = bestPaidPerTitlerawDF.withColumn("rank_salary", row_number().over(byTitle)).filter(col("rank_salary") <= 3)

bestPaidPerTitleDF.show(6)

+------+------------------+-----------+
|salary|             title|rank_salary|
+------+------------------+-----------+
|101622|Assistant Engineer|          1|
| 92674|Assistant Engineer|          2|
| 92034|Assistant Engineer|          3|
|130939|          Engineer|          1|
|121819|          Engineer|          2|
|120417|          Engineer|          3|
+------+------------------+-----------+
only showing top 6 rows



### UDFs

In [ ]:
charactersDF.show(3)

In [ ]:
# we define a function to remove the BBY from the birth_year
year = udf(lambda s: s[:-3])

charactersDF.withColumn("year", year(col("birth_year"))).show(3)

## Exercises Window Partitioning
1) Load characters.csv to a DataFrame. Then, get the (two) tallest characters per species and per homeworld planet. Select the name, height, and species/homeworld in their case.
2) Get the height difference for each character with respect to the smallest one in the same homeworld.

Exercise 1

Exercise 2

## Exercises UDFs
1. Choose one of the DFs we have, define two UDFs of your own, and apply them (with a withColumn) on the DF. Show the results 